In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

![](./media/AD_Banner.jpg)
<a id="top">

# openad-OpenToolkit Demonstration

## Working with OpenAD Magic Commands

When using Magic commands to access the Openad toolkit you have 2 options 

1. `%openad` provides a simple user interface that provides styled and formatted objects back to the notebook. Tables use pandas Dataframe Styler object. These can be converted back to data frame objects using `.data` on the object or using the in memory assistant which will copy the last result to a file , dataframe or to the dataviewer.
  When this is available you will see `Next up, you can run: result open/edit/copy/display/as dataframe/save [as '<filename.csv>']` in the output.
  
  This magic command is the recommended version to use as it willprovide all warning and results visually.
  
2. `%openadd` is the second form that allows you to return api style results in dataframe or list formats that can be used programatically for functions or flows in your notebook. This is good for prebuilt notebook process flows.

# ***What Commands Are Available  ?***

To learn what commands available you simply need to enter a `?` at the command line or at a magic command to list avialable commands

In [ ]:
%openad ?

# ***Individual Toolkits Help***

To look at the help purely for a specific toolkit, you simply need to ensure the toolkit is added then run the following.

%openad add toolkit rxn

In [ ]:
%openad ? rxn

In [ ]:
%openad clear sessions
%openad remove toolkit ds4sd
%openad remove toolkit rxn
%openad add toolkit ds4sd
%openad add toolkit rxn
%openad set context ds4sd

# ***Working Within the Open ToolKit Environment***

There are a Range of different commands that can be run to navigate through the toolkit, looking at `workspaces`, `toolkits`, `files` and `runs` within `workspaces` etc.<br> For example:

In [ ]:
%openad list workspaces

In [ ]:
%openad list files 

# ***Runs***

The concept of a run is to be able to re execute a consecutive set of commands in order, ideally that provide information to one another.

In [ ]:
%openad ? run

We are now going to create a run for demostration purposes.

In [ ]:
%openad create run

In [ ]:
%openad list all toolkits

In [ ]:
%openad set context rxn
%openad List rxn models

In [ ]:
%openad list runs

When we want to stop and record the run we simply execute the following.

In [ ]:
%openad save run as test_run3

We can display its contents with the following command.

In [ ]:
%openad display run test_run3

Now lets run it !

In [ ]:
%openad run test_run3

In [ ]:
%openad set llm bam

# ***Deep Search to RXN Demonstration***

In this Demonstration we are going to Search for a Smiles definition of a Drug in <b>Deep Search</b> and determine retrosynthesis Paths in <b>RXN</b>

First lets ask the `Tell Me` Me Functionality how we should do it.

In [ ]:
%openad tell me how to get the canonical smiles string for a molecule  and then what command to predict retrosynthesis

In [ ]:
mol = %openad @aspirin>>canonical_smiles

%openad set context ds4sd
%openad search for patents containing molecule '{mol}'

%openad set context rxn
%openad predict retrosynthesis '{mol}'

## ***Now lets Do a sequence of different commands and look at the results***

### ***Set the Tool Context to Deep Search***

In [ ]:
%openad set context ds4sd
target_smile= %openad @tritace>>canonical_smiles

In [ ]:
%openad search for patents containing molecule '{target_smile}'

### ***Let's look at what Domains and Collections exist for us to search***

In [ ]:
%openad  display all collections 

## Let's Look at The Collection we may find the most interesting

In [ ]:
%openad  display collection details 'Crossref'

Now we will Search for collections with a mention of the Molecule called  `Sibofimloc`, research it and find its SMILE returning Result to a data frame

What collections mention the molecule  'Sibofimloc' ?

In [ ]:
%openad  display collection matches for 'Sibofimloc'

### Lets Research the Drug for Safety and read the Papers where it is discused

In [ ]:
%openad search collection 'crossref' for 'Sibofimloc AND safety'    SHOW (docs data) 

### Lets Pull the Data for the Molecule <br>
by using the molecules Functions

In [ ]:
%openad display molecule Sibofimloc

In [ ]:
target_string=%openad @Sibofimloc>>canonical_smiles

### Let's have a look at similar molecules and Related Patents<br>

This will allows us also to look at similar targets we may want to pursue if our further experiements do not work as intended.

In [ ]:
similar_targets = %openad search for similar molecules to '{target_string}' save as 'similar_targets.csv'
display(similar_targets)

In [ ]:
mgrid = %openad show molecules using file 'similar_targets.csv' as molsobject

In [ ]:
mgrid.display()

In [ ]:
similar_targets = mgrid.get_selection()
similar_targets

In [ ]:
%openad search for patents containing molecule '{target_string}'

For each of those similar targets lets go and find the patents that they are mentioned in !

In [ ]:

for similar_mol  in similar_targets['SMILES']:
   my_patents=%openad search for patents containing molecule '{similar_mol}'
   %openad search for molecules in patents from dataframe  my_patents

In [ ]:
%openad display molecule CCC(C)C(=O)N1CCC2(CC1)c1cc(C#CC3OC(CO)C(O)C(O)C3O)ccc1-c1ccc(C#CC3OC(CO)C(O)C(O)C3O)cc12

### ***Set Context to the RXN Toolkit and run Retro Synthesis Prediction On our chosen Molecule***

Now with our Original target Molecule `Sibofimloc` lets perform retrosynthesis analyis to determine routes to snythesize the molecule.

In [ ]:
%openad set context rxn
%openad predict retrosynthesis '{target_string}' using( max_steps=3 )

In [ ]:
%openad add molecule {target_string}
%openad enrich molecules with analysis
%openad display molecule {target_string}

## Interpreting  Recipes

In [ ]:
%openad interpret recipe 'To a stirred solution of 7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one (110 mg, 0.42 mmol)\
in methanol (4 mL) was added sodium borohydride \
(24 mg, 0.62 mmol). The reaction mixture was stirred at ambient temperature for 1 hour.'

## Predicting Reaction Outcomes

### Single Predictions

In [ ]:
%openad  predict reaction 'Cl.CCC(=O)NCCC.O' use_saved

In [ ]:
%openad import  from '~/openad_notebooks/examples/reactions.csv' to 'reactions.csv'
%openad predict reaction in batch from file 'reactions.csv'

### Batch Predictions

In [ ]:
%openad predict reaction in batch from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 

### Using the [use_saved] clause

Providing the ability to reuse recently run previous reaction runs

In [ ]:
%openad predict reaction in batch  from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'BrBr.c1ccc2cc3ccccc3cc2c1','Cl.CCC(=O)NCCC.O'] use_saved

In [ ]:
%openad set context rxn
import pandas
lst = ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 
batch_list = pandas.DataFrame(lst,columns=['reactions'])
%openad predict reaction topn in batch from dataframe batch_list using (topn=3) use_saved

## Quickly investingation and adding to a set of molecules

In [ ]:
%openad add molecule tritace force
%openad add molecule aspirin force
%openad add molecule Ibuprofen force
%openad add molecule entresto force

In [ ]:
%openad list molecules

In [ ]:
 mol = %openad @tritace>>canonical_smiles
%openad set context rxn
%openad predict retrosynthesis '{mol}' using( max_steps=3 )

In [ ]:
%openad set context ds4sd
%openad search for patents containing molecule '{mol}'
%openad search for similar molecules to '{mol}'


In [ ]:
%openad list molecules

In [ ]:
%openad enrich molecules with analysis 
%openad display molecule tritace

In [ ]:
%openad list molecules